**New York City Taxi Ride Duration Prediction**

In this case study, we will build a predictive model to predict the duration of BUS ride. We will do the following steps:
  * Install the dependencies
  * Load the data as pandas dataframe
  * Define the outcome variable - the variable we are trying to predict.
  * Build features with Deep Feature Synthesis using the [featuretools](https://featuretools.com) package. We will start with simple features and incrementally improve the feature definitions and examine the accuracy of the system

In [1]:
!pip install featuretools==0.27.0

#### Note: If !pip install featuretools doesn't work, please install using the anaconda prompt by typing the following command in anaconda prompt
      conda install -c conda-forge featuretools==0.27.0

**Install Dependencies**
<p>If you have not done so already, download this repository <a href="https://github.com/Featuretools/DSx/archive/master.zip">from git</a>. Once you have downloaded this archive, unzip it and cd into the directory from the command line. Next run the command ``./install_osx.sh`` if you are on a mac or ``./install_linux.sh`` if you are on Linux. This should install all of the dependencies.</p>
<p> If you are on a windows machine, open the requirements.txt folder and make sure to install each of the dependencies listed (featuretools, jupyter, pandas, sklearn, numpy) </p>
<p> Once you have installed all of the dependencies, open this notebook. On Mac and Linux, navigate to the directory that you downloaded from git and run ``jupyter notebook`` to be taken to this notebook in your default web browser. When you open the NewYorkCity_taxi_case_study.ipynb file in the web browser, you can step through the code by clicking the ``Run`` button at the top of the page. If you have any questions for how to use <a href="http://jupyter.org/">Jupyter</a>, refer to Google or the discussion forum.</p>

### Importing libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Feataurestools for feature engineering
import featuretools as ft

#Utils python file contains some fuctions which can be used any where you want. 
import utils
from utils import load_nyc_taxi_data, compute_features, preview, feature_importances

# Importing gradient boosting regressor, to make prediction
from sklearn.ensemble import GradientBoostingRegressor

#importing primitives
from featuretools.primitives import (Minute, Hour, Day, Week, Month,
                                     Weekday, IsWeekend, Count, Sum, Mean, Median, Std, Min, Max)

print(ft.__version__)
%load_ext autoreload
%autoreload 2

0.27.0


### Step 1: Load the raw data

In [3]:
trips, pickup_neighborhoods, dropoff_neighborhoods = load_nyc_taxi_data()
preview(trips, 10)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,payment_type,trip_duration,pickup_neighborhood,dropoff_neighborhood
0,0,2,2016-01-01 00:00:19,2016-01-01 00:06:31,3,1.32,-73.961258,40.796200,-73.950050,40.787312,2,372.0,AH,C
672146,672146,1,2016-04-29 07:01:31,2016-04-29 07:15:46,1,3.30,-73.949951,40.784653,-73.982536,40.755470,1,855.0,C,AA
672147,672147,2,2016-04-29 07:01:43,2016-04-29 07:09:15,1,1.14,-73.967331,40.757370,-73.954277,40.765282,1,452.0,N,K
672148,672148,1,2016-04-29 07:01:46,2016-04-29 07:07:54,1,1.10,-74.003082,40.727509,-73.984703,40.724377,1,368.0,AB,AC
672149,672149,2,2016-04-29 07:01:46,2016-04-29 07:06:48,2,1.40,-73.990158,40.772350,-73.982147,40.759800,1,302.0,AR,AA
672150,672150,1,2016-04-29 07:01:59,2016-04-29 07:07:33,1,1.20,-73.983681,40.746677,-73.971703,40.762463,2,334.0,AO,A
672151,672151,2,2016-04-29 07:02:11,2016-04-29 07:15:24,2,2.13,-73.994209,40.750999,-73.969391,40.761539,1,793.0,D,AK
672152,672152,1,2016-04-29 07:02:11,2016-04-29 07:06:44,1,1.00,-73.983276,40.770985,-73.980110,40.760666,1,273.0,AR,A
672153,672153,2,2016-04-29 07:02:13,2016-04-29 07:08:36,1,1.17,-73.980141,40.743168,-73.983391,40.754665,1,383.0,Y,AA
672154,672154,1,2016-04-29 07:02:16,2016-04-29 07:04:07,1,0.50,-73.965973,40.765381,-73.970558,40.758724,1,111.0,AK,N


The ``trips`` table has the following fields
* ``id`` which uniquely identifies the trip
* ``vendor_id`` is the taxi cab company - in our case study we have data from three different cab companies
* ``pickup_datetime`` the time stamp for pickup
* ``dropoff_datetime`` the time stamp for drop-off
* ``passenger_count`` the number of passengers for the trip
* ``trip_distance`` total distance of the trip in miles 
* ``pickup_longitude`` the longitude for pickup
* ``pickup_latitude`` the latitude for pickup
* ``dropoff_longitude``the longitude of dropoff 
* ``dropoff_latitude`` the latitude of dropoff
* ``payment_type`` a numeric code signifying how the passenger paid for the trip. 1= Credit card 2= Cash 3= No charge 4= Dispute 5= Unknown 6= Voided
* ``trip_duration`` this is the duration we would like to predict using other fields 
* ``pickup_neighborhood`` a one or two letter id of the neighborhood where the trip started
* ``dropoff_neighborhood`` a one or two letter id of the neighborhood where the trip ended

### Step 2: Prepare the Data

Lets create entities and relationships. The three entities in this data are 
* trips 
* pickup_neighborhoods
* dropoff_neighborhoods

This data has the following relationships
* pickup_neighborhoods --> trips (one neighborhood can have multiple trips that start in it. This means pickup_neighborhoods is the ``parent_entity`` and trips is the child entity)
* dropoff_neighborhoods --> trips (one neighborhood can have multiple trips that end in it. This means dropoff_neighborhoods is the ``parent_entity`` and trips is the child entity)

In <a <href="https://www.featuretools.com/"><featuretools (automated feature engineering software package)/></a>, we specify the list of entities and relationships as follows: 


### Question 1: Define entities and relationships for the Deep Feature Synthesis (5 Marks)

In [4]:
entities = {
        "trips": (trips, "id", 'pickup_datetime' ),
        "pickup_neighborhoods": (pickup_neighborhoods, "neighborhood_id"),
        "dropoff_neighborhoods": (dropoff_neighborhoods, "neighborhood_id"),
        }

relationships = [("pickup_neighborhoods", "neighborhood_id", "trips", "pickup_neighborhood"),
                 ("dropoff_neighborhoods", "neighborhood_id", "trips", "dropoff_neighborhood")]

Next, we specify the cutoff time for each instance of the target_entity, in this case ``trips``.This timestamp represents the last time data can be used for calculating features by DFS. In this scenario, that would be the pickup time because we would like to make the duration prediction using data before the trip starts. 

For the purposes of the case study, we choose to only select trips that started after January 12th, 2016. 

In [5]:
cutoff_time = trips[['id', 'pickup_datetime']]
cutoff_time = cutoff_time[cutoff_time['pickup_datetime'] > "2016-01-12"]
preview(cutoff_time, 10)

,id,pickup_datetime
56311,56311,2016-01-12 00:00:25
698765,698765,2016-05-03 18:54:53
698766,698766,2016-05-03 18:55:37
698767,698767,2016-05-03 18:55:38
698768,698768,2016-05-03 18:55:49
698769,698769,2016-05-03 18:55:58
698770,698770,2016-05-03 18:56:22
698771,698771,2016-05-03 18:56:24
698772,698772,2016-05-03 18:56:51
698773,698773,2016-05-03 18:56:56


### Step 3: Create baseline features using Deep Feature Synthesis

Instead of manually creating features, such as "month of pickup datetime", we can let DFS come up with them automatically. It does this by 
* interpreting the variable types of the columns e.g categorical, numeric and others 
* matching the columns to the primitives that can be applied to their variable types
* creating features based on these matches

**Create transform features using transform primitives**

As we described in the video, features fall into two major categories, ``transform`` and ``aggregate``. In featureools, we can create transform features by specifying ``transform`` primitives. Below we specify a ``transform`` primitive called ``weekend`` and here is what it does:

* It can be applied to any ``datetime`` column in the data. 
* For each entry in the column, it assess if it is a ``weekend`` and returns a boolean. 

In this specific data, there are two ``datetime`` columns ``pickup_datetime`` and ``dropoff_datetime``. The tool automatically creates features using the primitive and these two columns as shown below. 

### Question 2: Create a model with only 1 transform primitive (10 Marks)

**Question: 2.1 Define transform primitive for weekend and define features using dfs?**

In [6]:
#defining Transform feature as weather the ride was at the weekend or not, and it is affecting the duration of the ride. 
trans_primitives = [IsWeekend]

#defining features we want to create using featuretools deep featurs synthesis(dfs)
features = ft.dfs(entities=entities,
                  relationships=relationships,
                  target_entity="trips",
                  trans_primitives=trans_primitives,
                  agg_primitives=[],
                  ignore_variables={"trips": ["pickup_latitude", "pickup_longitude",
                                              "dropoff_latitude", "dropoff_longitude"]},
                  features_only=True)

*If you're interested about parameters to DFS such as `ignore_variables`, you can learn more about these parameters [here](https://docs.featuretools.com/generated/featuretools.dfs.html#featuretools.dfs)*
<p>Here are the features created.</p>

In [7]:
print ("Number of features: %d" % len(features))
features

Number of features: 13


[<Feature: vendor_id>,
 <Feature: passenger_count>,
 <Feature: trip_distance>,
 <Feature: payment_type>,
 <Feature: trip_duration>,
 <Feature: pickup_neighborhood>,
 <Feature: dropoff_neighborhood>,
 <Feature: IS_WEEKEND(dropoff_datetime)>,
 <Feature: IS_WEEKEND(pickup_datetime)>,
 <Feature: pickup_neighborhoods.latitude>,
 <Feature: pickup_neighborhoods.longitude>,
 <Feature: dropoff_neighborhoods.latitude>,
 <Feature: dropoff_neighborhoods.longitude>]

Now let's compute the features. 

**Question: 2.2 Compute features and define feature matrix**

In [8]:
def compute_features(features, cutoff_time):
    # shuffle so we don't see encoded features in the front or backs

    np.random.shuffle(features)
    feature_matrix = ft.calculate_feature_matrix(features,
                                                 cutoff_time=cutoff_time,
                                                 approximate='36d',
                                                 verbose=True,entities=entities, relationships=relationships)
    print("Finishing computing...")
    feature_matrix, features = ft.encode_features(feature_matrix, features,
                                                  to_encode=["pickup_neighborhood", "dropoff_neighborhood"],
                                                  include_unknown=False)
    return feature_matrix

In [9]:
feature_matrix1 = compute_features(features, cutoff_time)

Elapsed: 00:06 | Progress: 100%|██████████
Finishing computing...


In [10]:
preview(feature_matrix1, 5)

,dropoff_neighborhood = AD,dropoff_neighborhood = A,dropoff_neighborhood = AA,dropoff_neighborhood = D,dropoff_neighborhood = AR,dropoff_neighborhood = C,dropoff_neighborhood = O,dropoff_neighborhood = N,dropoff_neighborhood = AO,dropoff_neighborhood = AK,...,pickup_neighborhood = AA,pickup_neighborhood = D,pickup_neighborhood = A,pickup_neighborhood = AR,pickup_neighborhood = AK,pickup_neighborhood = AO,pickup_neighborhood = N,pickup_neighborhood = R,pickup_neighborhood = O,IS_WEEKEND(dropoff_datetime)
id,,,,,,,,,,,,,,,,,,,,,
56311,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
691284,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
691285,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
691286,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
691288,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False


### Build the Model

To build a model, we
* Separate the data into a portion for ``training`` (75% in this case) and a portion for ``testing`` 
* Get the log of the trip duration so that a more linear relationship can be found.
* Train a model using a ``GradientBoostingRegressor``

**Question: 2.3 What was the Modeling Score after your last training round?**

**Question: 2.4 Hypothesize on how including more robust features will change the accuracy.**

In [11]:
# separates the whole feature matrix into train data feature matrix, 
# train data labels, and test data feature matrix 
X_train1, y_train1, X_test1, y_test1 = utils.get_train_test_fm(feature_matrix1,.75)
y_train1 = np.log(y_train1 +1)
y_test1 = np.log(y_test1 +1)

In [12]:
model1 = GradientBoostingRegressor(verbose=True)
model1.fit(X_train1, y_train1)
model1.score(X_test1, y_test1)

      Iter       Train Loss   Remaining Time 
         1           0.4925            1.97m
         2           0.4333            1.91m
         3           0.3843            1.89m
         4           0.3446            1.88m
         5           0.3119            1.86m
         6           0.2852            1.84m
         7           0.2634            1.81m
         8           0.2454            1.79m
         9           0.2305            1.77m
        10           0.2183            1.76m
        20           0.1666            1.57m
        30           0.1558            1.36m
        40           0.1514            1.16m
        50           0.1488           57.44s
        60           0.1472           45.57s
        70           0.1458           33.96s
        80           0.1448           22.60s
        90           0.1440           11.28s
       100           0.1433            0.00s


0.7220107526801756

**The score for the model with transform primitive is ~72.2% . The more robust features we add incrementally, we could see that the accuracy improves by observing the increasing modeling score.**

### Step 5: Adding more Transform Primitives

* Add ``Minute``, ``Hour``, ``Week``, ``Month``, ``Weekday`` , etc primitives
* All these transform primitives apply to ``datetime`` columns

### Question 3: Create a model with more transform primitives (10 Marks)

**3.1 Define more transform primitives and define features using dfs?**

In [13]:
trans_primitives = [Minute, Hour, Day, Week, Month, Weekday, IsWeekend]

features = ft.dfs(entities=entities,
                  relationships=relationships,
                  target_entity="trips",
                  trans_primitives=trans_primitives,
                  agg_primitives=[],
                  ignore_variables={"trips": ["pickup_latitude", "pickup_longitude",
                                              "dropoff_latitude", "dropoff_longitude"]},
                  features_only=True)

In [14]:
print ("Number of features: %d" % len(features))
features

Number of features: 25


[<Feature: vendor_id>,
 <Feature: passenger_count>,
 <Feature: trip_distance>,
 <Feature: payment_type>,
 <Feature: trip_duration>,
 <Feature: pickup_neighborhood>,
 <Feature: dropoff_neighborhood>,
 <Feature: DAY(dropoff_datetime)>,
 <Feature: DAY(pickup_datetime)>,
 <Feature: HOUR(dropoff_datetime)>,
 <Feature: HOUR(pickup_datetime)>,
 <Feature: IS_WEEKEND(dropoff_datetime)>,
 <Feature: IS_WEEKEND(pickup_datetime)>,
 <Feature: MINUTE(dropoff_datetime)>,
 <Feature: MINUTE(pickup_datetime)>,
 <Feature: MONTH(dropoff_datetime)>,
 <Feature: MONTH(pickup_datetime)>,
 <Feature: WEEK(dropoff_datetime)>,
 <Feature: WEEK(pickup_datetime)>,
 <Feature: WEEKDAY(dropoff_datetime)>,
 <Feature: WEEKDAY(pickup_datetime)>,
 <Feature: pickup_neighborhoods.latitude>,
 <Feature: pickup_neighborhoods.longitude>,
 <Feature: dropoff_neighborhoods.latitude>,
 <Feature: dropoff_neighborhoods.longitude>]

Now let's compute the features. 

**Question: 3.2 Compute features and define feature matrix**

In [15]:
feature_matrix2 = compute_features(features, cutoff_time)

Elapsed: 00:08 | Progress: 100%|██████████
Finishing computing...


In [16]:
preview(feature_matrix2, 10)

,WEEKDAY(pickup_datetime),DAY(dropoff_datetime),HOUR(dropoff_datetime),WEEK(dropoff_datetime),DAY(pickup_datetime),HOUR(pickup_datetime),pickup_neighborhood = AD,pickup_neighborhood = AA,pickup_neighborhood = D,pickup_neighborhood = A,...,trip_duration,IS_WEEKEND(dropoff_datetime),trip_distance,WEEK(pickup_datetime),MINUTE(dropoff_datetime),pickup_neighborhoods.longitude,dropoff_neighborhoods.latitude,vendor_id,WEEKDAY(dropoff_datetime),pickup_neighborhoods.latitude
id,,,,,,,,,,,,,,,,,,,,,
56311,1,12,0,2,12,0,False,False,False,False,...,645.0,False,1.61,2,11,-73.987205,40.721435,2,1,40.720245
691284,0,2,12,18,2,12,False,False,False,False,...,160.0,False,0.61,18,24,-73.991595,40.721435,2,0,40.729652
691285,0,2,12,18,2,12,False,False,False,False,...,295.0,False,0.88,18,27,-73.982322,40.785005,2,0,40.776270
691286,0,2,12,18,2,12,False,False,False,False,...,1573.0,False,1.90,18,48,-73.977943,40.757707,1,0,40.742531
691288,0,2,12,18,2,12,False,False,False,False,...,404.0,False,1.00,18,30,-73.985336,40.761087,1,0,40.747126
691289,0,2,12,18,2,12,False,False,False,False,...,1906.0,False,3.24,18,55,-73.998366,40.761492,2,0,40.721435
691290,0,2,12,18,2,12,False,False,False,False,...,156.0,False,0.10,18,26,-73.966696,40.764723,1,0,40.764723
691291,0,2,12,18,2,12,False,False,False,False,...,827.0,False,1.60,18,37,-73.956886,40.776270,1,0,40.766809
691292,0,2,12,18,2,12,True,False,False,False,...,883.0,False,1.50,18,39,-73.976515,40.764723,1,0,40.752186


### Step 6: Build the new model

**Question: 3.3 What was the Modeling Score after your last training round when including the transform primitives?**

**Question: 3.4 Comment on how the modeling accuracy differs when including more transform features.**

In [17]:
# separates the whole feature matrix into train data feature matrix,
# train data labels, and test data feature matrix 
X_train2, y_train2, X_test2, y_test2 = utils.get_train_test_fm(feature_matrix2,.75)
y_train2 = np.log(y_train2+1)
y_test2 = np.log(y_test2+1)

In [18]:
#remove ________ and write the code
model2 = GradientBoostingRegressor(verbose=True)
model2.fit(X_train2,y_train2)
model2.score(X_test2,y_test2)

      Iter       Train Loss   Remaining Time 
         1           0.4925            3.26m
         2           0.4333            3.23m
         3           0.3843            3.22m
         4           0.3444            3.21m
         5           0.3117            3.16m
         6           0.2848            3.13m
         7           0.2620            3.10m
         8           0.2435            3.07m
         9           0.2282            3.03m
        10           0.2152            2.99m
        20           0.1588            2.64m
        30           0.1415            2.27m
        40           0.1332            1.92m
        50           0.1283            1.59m
        60           0.1252            1.26m
        70           0.1227           56.10s
        80           0.1207           37.42s
        90           0.1191           18.63s
       100           0.1177            0.00s


0.7755557670208891

**As we can see the score for the model is better than previous one, and improved from ~72.2% to ~77.5% , and the time for computaito is not that long. we will check the model performance with aggregate primitive as well**

### Step 7: Add Aggregation Primitives

Now let's add aggregation primitives. These primitives will generate features for the parent entities ``pickup_neighborhoods``, and ``dropoff_neighborhood`` and then add them to the trips entity, which is the entity for which we are trying to make prediction.

### Question 4: Create a model with transform and aggregate primitive (10 Marks)
**4.1 Define more transform and aggregate primitive and define features using dfs?**

In [19]:
trans_primitives = [Minute, Hour, Day, Week, Month, Weekday, IsWeekend]
aggregation_primitives = [Count, Sum, Mean, Median, Std, Max, Min]

features = ft.dfs(entities=entities,
                  relationships=relationships,
                  target_entity="trips",
                  trans_primitives=trans_primitives,
                  agg_primitives=aggregation_primitives,
                  ignore_variables={"trips": ["pickup_latitude", "pickup_longitude",
                                              "dropoff_latitude", "dropoff_longitude"]},
                  features_only=True)

In [20]:
print ("Number of features: %d" % len(features))
features

Number of features: 63


[<Feature: vendor_id>,
 <Feature: passenger_count>,
 <Feature: trip_distance>,
 <Feature: payment_type>,
 <Feature: trip_duration>,
 <Feature: pickup_neighborhood>,
 <Feature: dropoff_neighborhood>,
 <Feature: DAY(dropoff_datetime)>,
 <Feature: DAY(pickup_datetime)>,
 <Feature: HOUR(dropoff_datetime)>,
 <Feature: HOUR(pickup_datetime)>,
 <Feature: IS_WEEKEND(dropoff_datetime)>,
 <Feature: IS_WEEKEND(pickup_datetime)>,
 <Feature: MINUTE(dropoff_datetime)>,
 <Feature: MINUTE(pickup_datetime)>,
 <Feature: MONTH(dropoff_datetime)>,
 <Feature: MONTH(pickup_datetime)>,
 <Feature: WEEK(dropoff_datetime)>,
 <Feature: WEEK(pickup_datetime)>,
 <Feature: WEEKDAY(dropoff_datetime)>,
 <Feature: WEEKDAY(pickup_datetime)>,
 <Feature: pickup_neighborhoods.latitude>,
 <Feature: pickup_neighborhoods.longitude>,
 <Feature: dropoff_neighborhoods.latitude>,
 <Feature: dropoff_neighborhoods.longitude>,
 <Feature: pickup_neighborhoods.COUNT(trips)>,
 <Feature: pickup_neighborhoods.MAX(trips.passenger_count)>

**Question: 4.2 Compute features and define feature matrix**

In [21]:
feature_matrix3 = compute_features(features, cutoff_time)

Elapsed: 00:20 | Progress: 100%|██████████
Finishing computing...


In [22]:
preview(feature_matrix3, 10)

,dropoff_neighborhoods.STD(trips.trip_duration),pickup_neighborhoods.STD(trips.passenger_count),pickup_neighborhoods.STD(trips.trip_distance),WEEK(pickup_datetime),dropoff_neighborhoods.MEDIAN(trips.trip_distance),pickup_neighborhoods.longitude,trip_duration,IS_WEEKEND(dropoff_datetime),pickup_neighborhoods.MEAN(trips.trip_distance),pickup_neighborhood = AD,...,dropoff_neighborhoods.SUM(trips.trip_duration),dropoff_neighborhoods.STD(trips.trip_distance),MINUTE(pickup_datetime),payment_type,dropoff_neighborhoods.MAX(trips.trip_duration),HOUR(pickup_datetime),pickup_neighborhoods.SUM(trips.passenger_count),MINUTE(dropoff_datetime),pickup_neighborhoods.STD(trips.trip_duration),pickup_neighborhoods.COUNT(trips)
id,,,,,,,,,,,,,,,,,,,,,
56311,487.621152,1.332149,2.518006,2,1.75,-73.987205,645.0,False,2.978551,False,...,1053070.0,2.714009,0,1,3572.0,0,2283.0,11,424.975281,1332.0
691284,520.080722,1.310267,2.099060,18,1.69,-73.991595,160.0,False,2.232347,False,...,13871368.0,2.477577,21,1,3603.0,12,34521.0,24,475.463994,20658.0
691285,481.137098,1.315427,2.111292,18,1.60,-73.982322,295.0,False,2.062772,False,...,12276557.0,2.461582,22,1,3602.0,12,36299.0,27,478.673953,21800.0
691286,593.180909,1.330233,2.137234,18,1.37,-73.977943,1573.0,False,2.125305,False,...,24003066.0,4.294641,22,1,3606.0,12,31158.0,48,444.617706,18764.0
691288,491.473671,1.319351,2.382495,18,1.50,-73.985336,404.0,False,2.171776,False,...,12359849.0,2.303356,23,1,3580.0,12,43543.0,30,475.990878,25905.0
691289,533.257705,1.315274,4.278999,18,1.40,-73.998366,1906.0,False,2.509054,False,...,24417513.0,2.598888,23,1,3606.0,12,30913.0,55,499.064139,18237.0
691290,456.642050,1.315487,1.846413,18,1.30,-73.966696,156.0,False,1.830726,False,...,14790756.0,1.843108,23,2,3580.0,12,43212.0,26,445.631332,26310.0
691291,465.303358,1.332776,2.206239,18,1.52,-73.956886,827.0,False,2.266902,False,...,14027834.0,4.096928,24,1,3604.0,12,32656.0,37,472.566001,19798.0
691292,456.642050,1.306579,2.488069,18,1.30,-73.976515,883.0,False,2.274509,True,...,14790756.0,1.843108,24,1,3580.0,12,57862.0,39,477.721891,35371.0


### Step 8: Build the new model

**Question 4.3 What was the Modeling Score after your last training round when including the aggregate transforms?**

**Question 4.4 How do these aggregate transforms impact performance? How do they impact training time?**

In [23]:
# separates the whole feature matrix into train data feature matrix,
# train data labels, and test data feature matrix 
X_train3, y_train3, X_test3, y_test3 = utils.get_train_test_fm(feature_matrix3,.75)
y_train3 = np.log(y_train3 + 1)
y_test3 = np.log(y_test3 + 1)

In [24]:
# note: this may take up to 20 minutes to run
model3 = GradientBoostingRegressor(verbose=True)
model3.fit(X_train3, y_train3)

model3.score(X_test3,y_test3)

      Iter       Train Loss   Remaining Time 
         1           0.4925            8.07m
         2           0.4333            7.95m
         3           0.3843            7.83m
         4           0.3444            7.72m
         5           0.3117            7.63m
         6           0.2848            7.54m
         7           0.2620            7.47m
         8           0.2435            7.38m
         9           0.2282            7.32m
        10           0.2152            7.24m
        20           0.1585            6.42m
        30           0.1420            5.62m
        40           0.1332            4.79m
        50           0.1271            3.97m
        60           0.1238            3.16m
        70           0.1211            2.36m
        80           0.1191            1.57m
        90           0.1176           47.02s
       100           0.1163            0.00s


0.7781928490478007

**The computation time was so long, yet the result is not that better than model 2,The score for the model with more aggregate transforms is ~77.8% and has improved only by a small amount. so it seems considering training time the best model is model 2 by using transform features.**

#### Based on the above 3 models, we can make predictions using our model2, as it is giving almost same accuracy as model3 and also the training time is not that large as compared to model3

In [25]:
y_pred = model2.predict(X_test2)
y_pred = np.exp(y_pred) - 1 # undo the log we took earlier
y_pred[5:]

array([ 553.50659125,  550.18452273, 1480.48337338, ..., 1106.04597122,
       1794.41849311,  789.39461652])

### Question 5: What are some important features based on model2 and how can they affect the duration of the rides? (5 Marks)

In [26]:
feature_importances(model2, feature_matrix2.columns, n=15)

1: Feature: IS_WEEKEND(dropoff_datetime), 0.894
2: Feature: HOUR(pickup_datetime), 0.027
3: Feature: HOUR(dropoff_datetime), 0.027
4: Feature: pickup_neighborhoods.longitude, 0.018
5: Feature: WEEKDAY(pickup_datetime), 0.008
6: Feature: dropoff_neighborhoods.longitude, 0.006
7: Feature: vendor_id, 0.004
8: Feature: WEEKDAY(dropoff_datetime), 0.003
9: Feature: MINUTE(dropoff_datetime), 0.002
10: Feature: dropoff_neighborhood = AA, 0.002
11: Feature: IS_WEEKEND(pickup_datetime), 0.002
12: Feature: payment_type, 0.001
13: Feature: MONTH(dropoff_datetime), 0.001
14: Feature: dropoff_neighborhood = A, 0.001
15: Feature: pickup_neighborhood = A, 0.001


**IS_WEEKEND(dropoff_datetime) is the most important feature, which implies that during weekend the rides are longer, specially dropoff (people usually don't drive after parties, bars, etc and take taxi).
Other important features is hour, it seems depending on the time (rush hour, late, etc) the duration of the rides might be longer.**